---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [45]:
import time
import pandas as pd
from calendar import month_name
import dateutil.parser
from datetime import datetime
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)


In [65]:
def date_sorter():
    df1 = df.str.extract('((?:\d?\d )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-,. ]\s?\d{0,2}[a-z]*[-,. ]?\s?\d{2,4}|\d{1,2}[\/-]\d{1,2}[\/-]\d{2,4}|(?:\d{1,2}[\/])\d{4}|(?:19|20)\d{2})')
    df11 = df1.str.replace(r'((\d?\d)\/(\d?\d)\/(\d\d))' , lambda x: x.groups()[2] +'/'+ x.groups()[1] +'/'+ x.groups()[3])
    #To add 19 for the dates which have only two digits for year eg:4/3/09
    df2 =df11.str.replace(r'^(\d{1,2}[-/])(\d{1,2}[-/])(\d{2})$' , lambda x: x.groups()[0]+x.groups()[1]+'19'+x.groups()[2])
    #To add January 1st in front of the dates which only have year eg:1980
    df3 = df2.str.replace(r'(^\d{4}$)',  lambda x: 'January 1,'+x.groups()[0])
    #To add 1 for the dates which have only month and year eg: 07/1996
    df4 = df3.str.replace(r'(^\d{1,2}/\d{4}$)', lambda x: '1/'+x.groups()[0])
    #To add 1 for dates which have only month and year eg January 1993 df4[298]
    df5 = df4.str.replace(r'^((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (\d{4})$', lambda x:  '1 '+ x.groups()[0]+','+x.groups()[2])   
    #Correct the typo in January spelling
    df5[298] = '1 January,1993'
    #To correct the typo in December spelling
    df5[313]= '1 December,1978'
    #To convert all the dates to the format %d/%m/%Y
    for i in range(len(df5)):
        df5[i] = dateutil.parser.parse(df5[i], dayfirst = True).strftime('%d/%m/%Y')
        df2[i] = time.mktime(datetime.strptime(df5[i], '%d/%m/%Y').timetuple())

    
    days = pd.DataFrame(df2)
    a = pd.DataFrame(days.sort_values(by = 0))    
    answer = pd.Series(a.index)
    return answer
    